In [1]:
from sqlalchemy import create_engine, text, bindparam, Integer
from sqlalchemy.orm import sessionmaker

import pandas as pd
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')
import pyodbc

In [55]:
class FaseToRuta:
    def __init__(self):
        self.server_solmicro = r'srvsql'
        self.server_industry = r'SERVIDOR'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.database_industry = 'IPFavram'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.password_industry = '71zl6p9h'
        self.connection_string_solmicro = create_engine(
            f'mssql+pyodbc://{self.username_solmicro}:{self.password_solmicro}@{self.server_solmicro}/{self.database_solmicro}?driver=SQL+Server')
        self.connection_string_industry = create_engine(
            f'mssql+pyodbc://{self.username_solmicro}:{self.password_industry}@{self.server_industry}/{self.database_industry}?driver=SQL+Server')
        self.connection = None
        self.connection_industry = None
        self.tables_info = []
        self.listArticulos = [443000021633, 442295636741, 443000021629, 443000022021, 443000022026, 443000022027, 443000021525, 443000021559, 443000021561, 443000021562, 443000021563,443000021565, 443000022076, 442296340551, 443000021582, 443000021583, 443000021584, 443000021585, 443000021627, 443000022005, 443000021626, 443000021625, 443000016122, 443000021628]
        self.connection_solmicro = f'DRIVER={{SQL Server}};SERVER={self.server_solmicro};DATABASE={self.database_solmicro};UID={self.username_solmicro};PWD={self.password_solmicro}'

    def Open_Solmicro(self):
        try:
            self.connection = pyodbc.connect(self.connection_solmicro)
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)
        


    def Open_Conn_Solmicro(self):
        try:
            self.connection = self.connection_string_solmicro.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def Open_Conn_Industry(self):
        try:
            self.connection = self.connection_string_industry.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def getDatosFase(self, listArticos):
        resultados = []
        try:
            conn = self.Open_Conn_Industry()
            if conn:
                for id in listArticos:
                    query = text(
                        f"SELECT Articulo, Fase, Descripcion, TipoFase, Centro, Ritmo, OperariosOperacion, OperariosPreparacion, TiempoPreparacion, TipoUtillaje, CodigoPreparacion, LoteMinimo, LoteLiberacion, FechaDeAlta, FechaUltimaModificacion,UsuarioAlta, UsuarioModificacion, Autocontrol, Version, GuardaVersion, UnidadesPorCiclo, TiempoCiclo, UnidadTiempoCiclo, IDDocumAdjuntos, TiempoHorasPieza, UnidadHorasPieza, CodigoMixProduction, Paralelo,RitmoCrono FROM MFase WHERE (Articulo ='{id}')")
                    result = conn.execute(query).fetchall()
                    if result:
                        resultados.append(result)
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            return resultados
    
    def RunAutonumeric(self,listArticos):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for dato in listArticos:
                    query = text("EXEC xAutonumericValue;")
                    conn.execute(query)
                    conn.commit()
        except Exception as e:
            print("Error ejecutando el procedimiento almacenado:", e)
            return None
        finally:
            if conn:
                conn.close()

    def GetAutonumericValue(self):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                query = text("SELECT autonumber FROM  ale_autoNumber")
                result = conn.execute(query).fetchall()
                final_result = [ value for row in result for value in row ]
                return final_result
        except Exception as e:
            print("Error ejecutando el procedimiento almacenado:", e)
            return None
        finally:
            if conn:
                conn.close()

    def GetPlazoSub(self,IDArticulco):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                query = text(f"SELECT SUM(DiasSub) FROM  vCTLCIRutaTiempo where IDArticulo = N'{IDArticulco}'")
                result = conn.execute(query).fetchall()
                return result[0] if result else 0
        except Exception as e:
            print("Error ejecutando GetPlazoSub:", e)
            return None
        finally:
            if conn:
                conn.close()
    
    def InsertIntotbRuta(self, datos,autonumeric):
        idx = 0
        conn = None
        result = []
        try:
            # conn = self.Open_Solmicro()
            cursor = 0
            if cursor == 0:
                for ruta in datos:
                    for fase in ruta:
                        IDRutaOp = autonumeric[idx]
                        IDArticulo = fase[0]
                        IDRuta = '01'
                        Secuencia = fase[1]
                        TipoOperacion = 1 if fase[4] == 450 else 0
                        IDOperacion = fase[1]                        
                        DescOperacion = rf"{fase[2]}"
                        Critica = 0
                        IDCentro = fase[4]+'0'
                        FactorHombre = 2 if fase[6] >2 and fase[6]< 990 else 1 if fase[6] < 2 else fase[5] 
                        IDUdProduccion = r'u.'
                        CantidadTiempo = fase[5]
                        UdTiempo = 1
                        FactorProduccion = 1
                        ControlProduccion = 1
                        TiempoPrep = 0
                        UdTiempoPrep = 2
                        TiempoEjecUnit = round(1 / CantidadTiempo,6) if CantidadTiempo != 0 else 0 
                        UdTiempoEjec = 1
                        FechaCreacionAudi = '2023-11-27 12:29:33.270'
                        FechaModificacionAudi = '2023-11-27 12:29:33.270'
                        UsuarioAudi = r'favram\a.obregon'
                        TiempoCiclo = fase[21]
                        UdTiempoCiclo = 1
                        LoteCiclo = 0
                        PlazoSub = self.GetPlazoSub(IDArticulco=fase[0])
                        UdTiempoPlazo = 1 if fase[4] != 450 else 0
                        SolapePor = 0
                        Ciclo = 0
                        Rendimiento = 100
                        IDCContable = 60400001 if fase[4] == 450 else 0
                        IdDocumentoEspecificacion = None
                        CantidadTiempo100 = 0
                        SolapeLote = 0
                        OcupacionMaquina = 0
                        Texto = ""
                        UsuarioCreacionAudi = ""
                        TiempoProgramacion = ""
                        IDTipoRuta = None
                        # print("Elemento: ",fase)
                        # stringInsert = f"insert into tbRuta (IDRutaOp, IDArticulo, IDRuta, Secuencia,TipoOperacion, IDOperacion, IDOperacion, Critica, IDCentro, FactorHombre, IDUdProduccion, CantidadTiempo, UdTiempo, FactorProduccion, ControlProduccion,TiempoPrep, UdTiempoPrep, TiempoEjecUnit, UdTiempoEjec, UsuarioAudi, TiempoCiclo, UdTiempoCiclo, LoteCiclo, PlazoSub, UdTiempoPlazo, SolapePor, Ciclo, Rendimiento,IDCContable, SolapeLote, OcupacionMaquina) values ({IDRutaOp}, N'{IDArticulo}',{IDRuta}, {Secuencia}, {TipoOperacion}, {IDOperacion},N'{DescOperacion}', 0, {IDCentro},{FactorHombre},{IDUdProduccion},{CantidadTiempo},{UdTiempo},N'{FactorProduccion}',N'{ControlProduccion}',N'{TiempoPrep}',N'{UdTiempoPrep}',N'{TiempoEjecUnit}',N'{UdTiempoEjec}',N'{UsuarioAudi}',{TiempoCiclo},{UdTiempoCiclo},{LoteCiclo},{PlazoSub},{UdTiempoPlazo},{SolapePor},0,{Rendimiento},{IDCContable},{SolapeLote},{OcupacionMaquina})"
                        # print(stringInsert)
                        # conn.execute(stringInsert)
                        # conn.commit()
                        result.append([IDRutaOp,IDArticulo,IDRuta,Secuencia,TipoOperacion,IDOperacion,DescOperacion,Critica,IDCentro,FactorHombre,IDUdProduccion,CantidadTiempo,UdTiempo,FactorProduccion,ControlProduccion,IDTipoRuta,TiempoPrep,UdTiempoPrep,TiempoEjecUnit,UdTiempoEjec,FechaCreacionAudi,FechaModificacionAudi,UsuarioAudi,TiempoCiclo,UdTiempoCiclo,LoteCiclo,PlazoSub,UdTiempoPlazo,SolapePor,Ciclo,Rendimiento,IDCContable,IdDocumentoEspecificacion,CantidadTiempo100,SolapeLote,OcupacionMaquina,Texto,UsuarioCreacionAudi,TiempoProgramacion])
                        idx = idx + 1
        except Exception as e:
            print("Error de insercion", e)
            return None
        # finally:
        #     if conn:
        #         cursor.close()
        return result
    
    def InsertTbArticuloRuta(self,articulos):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id in articulos:
                    stringInsert = text(f"insert into tbArticuloRuta (IDArticulo, IDRuta, DescRuta, Principal) values (N'{id}',N'01',N'Principal',{1})")
                    conn.execute(stringInsert)
                    conn.commit()
        except Exception as e:
            print("Error de insercion", e)
            return None
        finally:
            if conn:
                conn.close()

    def RunProcedure(self,datosExcel):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for dato in datosExcel:
                    for row in dato:
                        query = text("EXEC xAutonumericValue;")
                        conn.execute(query)
                        conn.commit()
        except Exception as e:
            print("Error ejecutando el procedimiento almacenado:", e)
            return None
        finally:
            if conn:
                conn.close()

    def GetAutonumericValue(self):
        conn = None
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                query = text("SELECT autonumber FROM  ale_autoNumber")
                result = conn.execute(query).fetchall()
                final_result = [ value for row in result for value in row ]
                return final_result
        except Exception as e:
            print("Error ejecutando el procedimiento almacenado:", e)
            return None
        finally:
            if conn:
                conn.close()

    @staticmethod
    def export_to_excel(data):
        df = pd.DataFrame(data,columns=["IDRutaOp","IDArticulo","IDRuta","Secuencia","TipoOperacion","IDOperacion","DescOperacion","Critica","IDCentro", "FactorHombre", "IDUdProduccion", "CantidadTiempo", "UdTiempo", "FactorProduccion", "ControlProduccion", "IDTipoRuta","TiempoPrep", "UdTiempoPrep", "TiempoEjecUnit", "UdTiempoEjec", "FechaCreacionAudi","FechaModificacionAudi", "UsuarioAudi", "TiempoCiclo", "UdTiempoCiclo", "LoteCiclo", "PlazoSub", "UdTiempoPlazo", "SolapePor", "Ciclo", "Rendimiento", "IDCContable","IdDocumentoEspecificacion", "CantidadTiempo100", "SolapeLote", "OcupacionMaquina", "Texto", "UsuarioCreacionAudi", "TiempoProgramacion"])
        df.to_excel("FaseToRuta.xlsx",index=False)
        

In [56]:
iFaseToRuta = FaseToRuta()

In [57]:
listSet = list(set(iFaseToRuta.listArticulos) - set([443000022021, 443000016122]) - set([443000021633, 442295636741, 443000021629]))

In [40]:
iFaseToRuta.InsertTbArticuloRuta(listSet)

In [58]:
datos = iFaseToRuta.getDatosFase(listArticos=listSet)

In [59]:
iFaseToRuta.RunProcedure(datosExcel=datos)

In [60]:
autonumeric = iFaseToRuta.GetAutonumericValue()

In [61]:
data = iFaseToRuta.InsertIntotbRuta(datos=datos,autonumeric=autonumeric)

In [62]:
iFaseToRuta.export_to_excel(data=data)

In [17]:
columnas_dataframe = ["IDRutaOp","IDArticulo","IDRuta","Secuencia","TipoOperacion","IDOperacion","DescOperacion","Critica","IDCentro","FactorHombre","IDUdProduccion","CantidadTiempo","UdTiempo","FactorProduccion","ControlProduccion","IDTipoRuta","TiempoPrep","UdTiempoPrep","TiempoEjecUnit","UdTiempoEjec","FechaCreacionAudi","FechaModificacionAudi","UsuarioAudi","TiempoCiclo","UdTiempoCiclo","LoteCiclo","PlazoSub","UdTiempoPlazo","SolapePor","Ciclo","Rendimiento","IDCContable","IdDocumentoEspecificacion","CantidadTiempo100","SolapeLote","OcupacionMaquina","Texto","UsuarioCreacionAudi","TiempoProgramacion"]

# Lista de nombres de las columnas presentes en los datos
columnas_datos = ["IDRutaOp","IDArticulo","IDRuta","Secuencia","TipoOperacion","IDOperacion","DescOperacion","Critica","IDCentro", "FactorHombre", "IDUdProduccion", "CantidadTiempo", "UdTiempo", "FactorProduccion", "ControlProduccion", "IDTipoRuta","TiempoPrep", "UdTiempoPrep", "TiempoEjecUnit", "UdTiempoEjec", "FechaCreacionAudi","FechaModificacionAudi", "UsuarioAudi", "TiempoCiclo", "UdTiempoCiclo", "LoteCiclo", "PlazoSub", "UdTiempoPlazo", "SolapePor", "Ciclo", "Rendimiento", "IDCContable","IdDocumentoEspecificacion", "CantidadTiempo100", "SolapeLote", "OcupacionMaquina", "Texto", "UsuarioCreacionAudi", "TiempoProgramacion"]

# Encontrar las columnas que están en el DataFrame pero no están en los datos
columnas_faltantes = set(columnas_datos) -set(columnas_dataframe) 
print(len(columnas_dataframe) )
print(len(columnas_dataframe))

# Imprimir las columnas faltantes
print("Columnas faltantes en los datos:", columnas_faltantes)

39
39
Columnas faltantes en los datos: set()
